# Target : http://127.0.0.1:4000/2021/03/20/Fluid-Typography-with-CSS-Clamp()-is-My-New-Favorite-Thing-DEV-Community.html
# File Path : 2021-03-20-Fluid-Typography-with-CSS-Clamp()-is-My-New-Favorite Thing---DEV-Community.md
# Title : Fluid Typography with CSS-Clamp() is My New Favorite Thing

In [18]:
from selenium import webdriver
from bs4 import BeautifulSoup
from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import Neo4jError
import csv
import json
import time
import os
import re
FOLDER_PATH = '../web/_posts/'
test_url = 'https://dev.to/cruip/50-free-tools-and-resources-to-create-awesome-user-interfaces-1c1b'
DATA_FILE_PATH = 'dev-to-articles.csv'
DATABASE_USERNAME="neo4j"
DATABASE_PASSWORD="spade-discounts-switch"
DATABASE_URL="bolt://localhost:7687"
DATA_FILE_PATH = 'dev-to-articles.csv'
driver = GraphDatabase.driver(DATABASE_URL, auth=basic_auth(DATABASE_USERNAME, str(DATABASE_PASSWORD)))

def cprint(content,module='DEBUG',*args):
    if args:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + '\033[1;35m' +str(args) +' \033[0m' + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()) )
    else:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()))

def remove_invalid_text(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]"  # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, "", title) 
    return new_title
def init_yaml_header(soup):
    '''
    Args: soup of source post
    Return: valid yaml for riinosite3
    '''
    title = soup.head.title.string
    date = soup.find("time", {"class":"date"})['datetime'][:10]
    author = soup.find("a",{"class":"flex items-center mr-4 mb-4 s:mb-0 fw-medium crayons-link"}).contents[-1].replace('\n','').strip()
    yaml = ['---\n',
            'layout: post\n'
            f'title: "{title}"\n',
            f'author: "{author}"\n',
            f'date: {date}\n',
            'toc: false\n'
            'tags:\n'
           ]
    for tag in soup.find_all("a",{"class":"crayons-tag"}):
        yaml.append('    - '+tag.text[1:]+'\n')
    yaml.append('---\n')
    return yaml
def generate_post_filename(published_at,title):
    '''
    Args: soup of source post
    Return: valid file name for riinosite3
    '''
    filename =  published_at[:10]+'-'+title.replace(' ','-')+'---DEV-Community'+'.md'
    filename = remove_invalid_text(filename)
    filename = filename.replace('.','-')
    return filename
def save_markdown_file(soup,folder_path):
    '''
    Args: save markdown file from soup into target folder path
    Return: None
    '''
    with open(FOLDER_PATH+generate_post_filename(soup), mode='w',encoding="utf-8") as file_w:
        #write yaml
        file_w.writelines(init_yaml_header(soup))
        #write body
        for i in soup.find("div", {"id":"article-body"}).contents:
            file_w.write(str(i))
        cprint('Write file ssuccessfully ','FILE',FOLDER_PATH+generate_post_filename(soup))
    file_w.close()

def db_update_article_jekyll_path(session,id,value):
    '''
    Add a new value for article node with id
    Args:
        session: db session,driver.session()
        id: article id, in csv file.
        value: value of that key : reading_time
    '''
    def _cypher(tx,key,value):
        return list(tx.run(
        '''
        MATCH (n:Article { id: $id })
        SET n.jekyll_path = $value
        RETURN n
        '''
        ))
    result = session.write_transaction(_cypher,key,value)
    return result

article_props = []

if os.path.exists(DATA_FILE_PATH):
    if not os.path.getsize(DATA_FILE_PATH):
        cprint(DATA_FILE_PATH +'is empty')
    else:
        with open(DATA_FILE_PATH, mode='r',encoding="utf-8") as data_file_r:
            csv_reader = csv.DictReader(data_file_r)
            line_count = 0
            props=set()
            for row in csv_reader:
                if line_count == 0:
                    cprint(f'Processing CSV header {", ".join(row)}','CSV')
                    line_count += 1
                article_prop = {
                            'id': row['id'],
                            'title': row['title'],
                            'url': row['url'],
                            'main_image_url': row['main_image_url'],
                            'reading_time': row['reading_time'],
                            'tag_names': row['tag_names'],
                            'published_at': row['published_at'],
                            'source_site':'dev.to',
                            'author_name':row['author_name'],
                            'count':row['public_reactions_count']
                }
                if article_prop in article_props:
                    continue
                else:
                    article_props.append(article_prop)
                line_count += 1
            cprint(f'File processed successfully with {line_count-1} ids.','CSV')
        data_file_r.close()
else:
    cprint(DATA_FILE_PATH +' does not exist')

 [CSV]  Processing CSV header id, title, url, main_image_url, reading_time, author_name, author_username, author_id, published_at, tag_names, keywords_for_search, comments_count, public_reactions_count, highlight |2021-05-09 20:31:52|
 [CSV]  File processed successfully with 2375 ids. |2021-05-09 20:31:52|


In [13]:
generate_post_filename(article_props[1]['published_at'],article_props[1]['title'])

'2020-10-06-ReactJS-Roadmap-🗺-For-Developers.💻---DEV-Community.md'

In [20]:
FOLDER_PATH+generate_post_filename(article_props[1]['published_at'],article_props[1]['title'])

'../web/_posts/2020-10-06-ReactJS-Roadmap-🗺-For-Developers-💻---DEV-Community-md'

In [ ]:
'2020-10-06-ReactJS-Roadmap-🗺-For-Developers-💻---DEV-Community.md'